<a href="https://colab.research.google.com/github/chinedudev9/AI-Based-Predictive-Marketplace-for-NFTs-or-Tokens/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [89]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:
print(nltk.corpus.stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [91]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Data pre-processing

In [92]:
fakeNews_data = pd.read_csv('/content/FakeNewsNet.csv')

In [93]:
fakeNews_data.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [94]:
fakeNews_data.describe()

,tweet_num,real
count,23196.000000,23196.000000
mean,88.956803,0.751897
std,488.694592,0.431921
min,0.000000,0.000000
25%,11.000000,1.000000
50%,37.000000,1.000000
75%,65.000000,1.000000
max,29060.000000,1.000000


In [95]:
#checking the number of missing value
fakeNews_data.isnull().sum()

,0
title,0
news_url,330
source_domain,330
tweet_num,0
real,0


In [96]:
# filling the missing data with empty string
fakeNews_data = fakeNews_data.fillna('')

In [97]:
#Now there is no missing data
fakeNews_data.isnull().sum()

,0
title,0
news_url,0
source_domain,0
tweet_num,0
real,0


In [98]:
# checking the size the data
fakeNews_data.shape

(23196, 5)

In [99]:
# concatenating the two roles (title and news_url) make a new dataset for the predictions
fakeNews_data['content'] = fakeNews_data['title'] + ' ' + fakeNews_data['news_url']

In [100]:
# print the content data
print(fakeNews_data['content'])

0        Kandi Burruss Explodes Over Rape Accusation on...
1        People's Choice Awards 2018: The best red carp...
2        Sophia Bush Sends Sweet Birthday Message to 'O...
3        Colombian singer Maluma sparks rumours of inap...
4        Gossip Girl 10 Years Later: How Upper East Sid...
                               ...                        
23191    Pippa Middleton wedding: In case you missed it...
23192    Zayn Malik & Gigi Hadid’s Shocking Split: Why ...
23193    Jessica Chastain Recalls the Moment Her Mother...
23194    Tristan Thompson Feels "Dumped" After Khloé Ka...
23195    Kelly Clarkson Performs a Medley of Kendrick L...
Name: content, Length: 23196, dtype: object


In [101]:
#separating the data from the label

X = fakeNews_data.drop(columns= 'real', axis=1)
Y = fakeNews_data['real']

In [102]:
# print X and Y
print(X)
print(Y)

                                                   title  ...                                            content
0      Kandi Burruss Explodes Over Rape Accusation on...  ...  Kandi Burruss Explodes Over Rape Accusation on...
1      People's Choice Awards 2018: The best red carp...  ...  People's Choice Awards 2018: The best red carp...
2      Sophia Bush Sends Sweet Birthday Message to 'O...  ...  Sophia Bush Sends Sweet Birthday Message to 'O...
3      Colombian singer Maluma sparks rumours of inap...  ...  Colombian singer Maluma sparks rumours of inap...
4      Gossip Girl 10 Years Later: How Upper East Sid...  ...  Gossip Girl 10 Years Later: How Upper East Sid...
...                                                  ...  ...                                                ...
23191  Pippa Middleton wedding: In case you missed it...  ...  Pippa Middleton wedding: In case you missed it...
23192  Zayn Malik & Gigi Hadid’s Shocking Split: Why ...  ...  Zayn Malik & Gigi Hadid’s Shockin

In [103]:
#Stemming: is the process of reducing a word to its root words. Example axtor and actress reduced to act
port_stem = PorterStemmer()

In [104]:
# defining the function that handle the stem
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [105]:
fakeNews_data['content'] = fakeNews_data['content'].apply(stemming)

In [106]:
print(fakeNews_data['content'])

0        kandi burruss explod rape accus real housew at...
1        peopl choic award best red carpet look http ww...
2        sophia bush send sweet birthday messag one tre...
3        colombian singer maluma spark rumour inappropr...
4        gossip girl year later upper east sider shock ...
                               ...                        
23191    pippa middleton wed case miss pippa marri lace...
23192    zayn malik gigi hadid shock split chanc reunit...
23193    jessica chastain recal moment mother boyfriend...
23194    tristan thompson feel dump khlo kardashian ref...
23195    kelli clarkson perform medley kendrick lamar h...
Name: content, Length: 23196, dtype: object


In [107]:
# separating the output data from the input data
X = fakeNews_data['content'].values
Y = fakeNews_data['real'].values

In [108]:
#Print X
print(X)

['kandi burruss explod rape accus real housew atlanta reunion video http toofab com real housew atlanta kandi burruss rape phaedra park porsha william'
 'peopl choic award best red carpet look http www today com style see peopl choic award red carpet look'
 'sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva http www etonlin com news sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva'
 ...
 'jessica chastain recal moment mother boyfriend slap kick genit http www justjar com jessica chastain recal moment mother boyfriend slap kick genit'
 'tristan thompson feel dump khlo kardashian refus let move la home exclus www intouchweekli com post tristan thompson feel dump khloe kardashian'
 'kelli clarkson perform medley kendrick lamar humbl hit billboard music award http www billboard com articl news bbma kelli clarkson medley bbma']


In [109]:
# Print Y
print(Y)

[1 1 1 ... 1 0 1]


In [110]:
Y.shape

(23196,)

In [111]:
# converting X to vectorizer so the system can understand because the value of X is in textual data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [112]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 316924 stored elements and shape (23196, 17314)>
  Coords	Values
  (0, 175)	0.14434726829473674
  (0, 896)	0.3389455172701989
  (0, 2218)	0.4346705393896895
  (0, 3144)	0.028363875555072016
  (0, 5230)	0.19914481139678233
  (0, 7204)	0.27967207824608226
  (0, 7240)	0.03029315079920102
  (0, 8136)	0.4346705393896895
  (0, 11301)	0.16947275863509945
  (0, 11540)	0.2113369959982094
  (0, 11846)	0.21414819505876584
  (0, 12356)	0.33456306448701023
  (0, 12418)	0.2501629274621897
  (0, 12736)	0.1394415242605404
  (0, 15510)	0.1652746695506423
  (0, 16345)	0.1037611672890413
  (0, 16828)	0.12458641416073078
  (1, 985)	0.33310329340670436
  (1, 1532)	0.18067383197494744
  (1, 2474)	0.4135533362491038
  (1, 2851)	0.4642049540481571
  (1, 3144)	0.04487540286284784
  (1, 7240)	0.04792777148735079
  (1, 9039)	0.36163482558348325
  (1, 11465)	0.25473967851260193
  :	:
  (23194, 8547)	0.17169083170481492
  (23194, 8810)	0.211463587763149

Splitting the data set into training and testing

In [113]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [114]:
model = LogisticRegression() # Changed model to LogisticRegression

In [115]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluating the Model

In [116]:
# Calculating the accuracy score training data
X_train_prediction = model.predict(X_train)
# Round the predictions to binary values for classification metrics
X_train_prediction = np.round(X_train_prediction).astype(int)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [117]:
# print accuracy score of the training data

print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9483724940719983


In [118]:
# Calculating the accuracy score of the test data
X_test_prediction = model.predict(X_test)
# Round the predictions to binary values for classification metrics
X_test_prediction = np.round(X_test_prediction).astype(int)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [119]:
# print accuracy score of the test data

print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9269396551724138


Making a Predictive System

In [131]:
X_new = X_test[0]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
    print('The News is Real')
else:
    print('The News is Fake')

[1]
The News is Fake


In [130]:
print(Y_test[1])

0
